<a href="https://colab.research.google.com/github/zahraniayudyaa/midterm-machine-learning/blob/main/Zahrani_UTS_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost==2.0.3 polars
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import polars as pl

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer

In [ ]:
!pip install -q gdown

# kalian download semua file dari gdrive dibawah lalu buat folder (ex. midterm_folder)
!gdown --folder 1qoyfQ0yXp9pgnp9zKwsRm0B-u0n-KopN -O midterm_folder

Retrieving folder contents
Processing file 1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk test_transaction.csv
Processing file 18BxWO8J8QgLfWeFInaLnswSph4yNg2RL train_transaction.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk
From (redirected): https://drive.google.com/uc?id=1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk&confirm=t&uuid=d51aeec0-e671-482c-833b-8edfc7fe4e57
To: /content/midterm_folder/test_transaction.csv
100% 613M/613M [00:07<00:00, 79.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=18BxWO8J8QgLfWeFInaLnswSph4yNg2RL
From (redirected): https://drive.google.com/uc?id=18BxWO8J8QgLfWeFInaLnswSph4yNg2RL&confirm=t&uuid=78cdcaeb-36f2-4afa-a6c1-cafea7f2eea9
To: /content/midterm_folder/train_transaction.csv
100% 683M/683M [00:05<00:00, 126MB/s]
Download completed


In [ ]:
BASE_PATH = "midterm_folder"

train_transaction = pl.read_csv(f"{BASE_PATH}/train_transaction.csv")
test_transaction  = pl.read_csv(f"{BASE_PATH}/test_transaction.csv")

# print(train_transaction.shape) #590540 row dan 393 feature + 1 target
# print(test_transaction.shape) #506691 row dan 393 feature

In [ ]:
train_transaction.head() #isFraud adalah target featurenya

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,"""W""",13926,null,150.0,"""discover""",142.0,"""credit""",315.0,87.0,19.0,null,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,"""W""",2755,404.0,150.0,"""mastercard""",102.0,"""credit""",325.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,"""W""",4663,490.0,150.0,"""visa""",166.0,"""debit""",330.0,87.0,287.0,null,"""outlook.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,"""W""",18132,567.0,150.0,"""mastercard""",117.0,"""debit""",476.0,87.0,null,null,"""yahoo.com""",null,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,"""H""",4497,514.0,150.0,"""mastercard""",102.0,"""credit""",420.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_transaction.head() #uji data baru, setelah pembuatan model machine learningnya

TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3663549,18403224,31.95,"""W""",10409,111.0,150.0,"""visa""",226.0,"""debit""",170.0,87.0,1.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,null,null,…,0.0,0.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663550,18403263,49.0,"""W""",4272,111.0,150.0,"""visa""",226.0,"""debit""",299.0,87.0,4.0,null,"""aol.com""",null,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,null,null,…,0.0,0.0,1.0,0.0,280.0,77.0,0.0,280.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663551,18403310,171.0,"""W""",4476,574.0,150.0,"""visa""",226.0,"""debit""",472.0,87.0,2635.0,null,"""hotmail.com""",null,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,null,null,…,0.0,0.0,1.0,0.0,1321.0,0.0,0.0,1058.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663552,18403310,284.95,"""W""",10989,360.0,150.0,"""visa""",166.0,"""debit""",205.0,87.0,17.0,null,"""gmail.com""",null,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663553,18403317,67.95,"""W""",18018,452.0,150.0,"""mastercard""",117.0,"""debit""",264.0,87.0,6.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,null,null,…,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
!pip install polars scikit-learn matplotlib scipy seaborn

# Tentukan nama kolom target
label_col = "isFraud"   # GANTI sesuai dataset Anda

# =======================================
# 2. DATA PREPROCESSING
# =======================================

# (A) Isi missing values
num_cols_train = [c for c, dt in zip(train_transaction.columns, train_transaction.dtypes)
            if dt in (pl.Float64, pl.Float32, pl.Int64, pl.Int32)]

cat_cols_train = [c for c, dt in zip(train_transaction.columns, train_transaction.dtypes)
            if dt == pl.Utf8]

num_cols_test = [c for c, dt in zip(test_transaction.columns, test_transaction.dtypes)
            if dt in (pl.Float64, pl.Float32, pl.Int64, pl.Int32)]

cat_cols_test = [c for c, dt in zip(test_transaction.columns, test_transaction.dtypes)
            if dt == pl.Utf8]


In [ ]:
train_transaction = train_transaction.with_columns([
    pl.col(num_cols_train).fill_null(strategy="mean"),
    pl.col(cat_cols_train).fill_null("missing")
])

test_transaction = test_transaction.with_columns([
    pl.col(num_cols_test).fill_null(strategy="mean"),
    pl.col(cat_cols_test).fill_null("missing")
])

In [ ]:
# (B) Pisahkan fitur & label dari TRAIN
feature_cols = [c for c in train_transaction.columns if c != label_col]

train_features = train_transaction.select(feature_cols)
train_label    = train_transaction.select(label_col)

# Untuk test set (tanpa label)
test_features = test_transaction.select(feature_cols)

In [ ]:
# ============================================
# 3. FEATURE SEPARATION (tanpa numpy)
# ============================================
feature_cols = [c for c in train_transaction.columns if c != label_col]

X= train_transaction.select(feature_cols).to_pandas()
y = train_transaction.select(label_col).to_pandas().values.ravel()

X_test = test_transaction.select(feature_cols).to_pandas()

In [ ]:
# =======================================
# 4. TRAIN-TEST SPLIT (VALIDASI)
# =======================================
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# print("Train size:", X_train.shape)
# print("Valid size:", X_valid.shape)

In [ ]:
!pip install optuna
import optuna
import xgboost as xgb
from sklearn.metrics import roc_auc_score

def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "tree_method": "gpu_hist",        # <— aktifkan GPU
        "predictor": "gpu_predictor",     # <— aktifkan GPU
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 5),
        "n_estimators": trial.suggest_int("n_estimators", 200, 1200),
    }

    model = xgb.XGBClassifier(**params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False
    )

    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)

    return auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

best_params = study.best_params
print("Best Params:", best_params)

best_model = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="gpu_hist",
    predictor="gpu_predictor"
)

best_model.fit(X_train, y_train)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 10.9 MB/s eta 0:00:00


[I 2025-12-02 07:17:08,016] A new study created in memory with name: no-name-941c5d4e-edc5-4081-9035-e33977e6d9f7
[W 2025-12-02 07:17:08,228] Trial 0 failed with parameters: {'learning_rate': 0.0688170448126806, 'max_depth': 12, 'min_child_weight': 5, 'subsample': 0.8145298875365685, 'colsample_bytree': 0.809883748346326, 'gamma': 2.6570732847254606, 'reg_alpha': 1.4422113245675616, 'reg_lambda': 3.9243776789705316, 'n_estimators': 931} because of the following error: ValueError('DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:ProductCD: object, card4: object, card6: object, P_emaildomain: object, R_emaildomain: object, M1: object, M2: object, M3: object, M4: object, M5: object, M6: object, M7: object, M8: object, M9: object').
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:ProductCD: object, card4: object, card6: object, P_emaildomain: object, R_emaildomain: object, M1: object, M2: object, M3: object, M4: object, M5: object, M6: object, M7: object, M8: object, M9: object

In [ ]:
# =======================================
# 6. EVALUATION ON VALID SET
# =======================================
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

best_model = random_search.best_estimator_

y_pred = best_model.predict(X_valid)

print("Validation Accuracy:", accuracy_score(y_valid, y_pred))
print("\nClassification Report:\n", classification_report(y_valid, y_pred))

plt.figure(figsize=(6,4))
sns.heatmap(confusion_matrix(y_valid, y_pred), annot=True, fmt="d")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


# =======================================
# 7. FINAL PREDICTION ON test_transaction.csv
# =======================================
final_predictions = best_model.predict(X_test_final)

print("Final predictions shape:", final_predictions.shape)
print(final_predictions[:20])  # preview

In [ ]:
!pip install polars optuna xgboost --quiet

import polars as pl
import xgboost as xgb
import optuna
from sklearn.metrics import roc_auc_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
# ============================
# 1. LOAD DATASET (POLARS)
# ============================
# train = pl.read_csv("/content/train_transaction.csv")
# test  = pl.read_csv("/content/test_transaction.csv")

# print("Train:", train.shape)
# print("Test :", test.shape)

In [ ]:
# ============================
# 2. HANDLE MISSING VALUES
# Pisah numeric & string
# ============================
def fill_null(df):
    num_cols = [c for c in df.columns if df[c].dtype.is_numeric()]
    str_cols = [c for c in df.columns if df[c].dtype == pl.String]

    return df.with_columns([
        pl.col(num_cols).fill_null(0),
        pl.col(str_cols).fill_null("unknown")
    ])

train_transaction = fill_null(train_transaction)
test_transaction  = fill_null(test_transaction)

In [ ]:
# ============================
# 3. EXPORT KE LIBSVM FORMAT
# (ANTI CRASH, DIPAKAI XGBOOST)
# ============================

def save_libsvm(df, target, file_out):
    features = [c for c in df.columns if c != target]

    with open(file_out, "w") as f:
        for row in df.iter_rows(named=True):
            label = row[target]
            line = str(int(label))  # XGBoost perlu numeric label

            for idx, feat in enumerate(features):
                val = row[feat]
                if val is None:
                    continue
                # skip categorical string → encode simple hash
                if isinstance(val, str):
                    val = hash(val) % 1_000_000
                line += f" {idx}:{val}"
            f.write(line + "\n")


# TRAIN LIBSVM
save_libsvm(train_transaction, "isFraud", "train.svm")

# TEST LIBSVM (label dummy = 0)
test_transaction = test_transaction.with_columns(pl.lit(0).alias("isFraud"))
save_libsvm(test_transaction, "isFraud", "test.svm")

In [ ]:
# ============================
# 4. EXTERNAL MEMORY DMATRIX
# (DATA TIDAK MASUK RAM)
# ============================

dtrain = xgb.DMatrix("train.svm?format=libsvm#dtrain.cache")
dtest  = xgb.DMatrix("test.svm?format=libsvm#dtest.cache")

XGBoostError: [10:35:20] /workspace/src/data/data.cc:923: Check failed: dlm_pos == std::string::npos: External memory training with text input has been removed.
Stack trace:
  [bt] (0) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x2bdf8c) [0x7d72494bdf8c]
  [bt] (1) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x57f193) [0x7d724977f193]
  [bt] (2) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(XGDMatrixCreateFromURI+0xfe) [0x7d72493cdb6e]
  [bt] (3) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7d72ce295e2e]
  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7d72ce292493]
  [bt] (5) /usr/lib/python3.12/lib-dynload/_ctypes.cpython-312-x86_64-linux-gnu.so(+0x98c1) [0x7d72cf4cc8c1]
  [bt] (6) /usr/lib/python3.12/lib-dynload/_ctypes.cpython-312-x86_64-linux-gnu.so(+0x8ffe) [0x7d72cf4cbffe]
  [bt] (7) /usr/bin/python3(_PyObject_MakeTpCall+0x2fb) [0x53f5db]
  [bt] (8) /usr/bin/python3(_PyEval_EvalFrameDefault+0x701) [0x54a

In [ ]:
# ============================
# 5. OPTUNA GPU TUNING
# ============================

y_train_np = train_transaction["isFraud"].to_numpy()   # kecil, aman

def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "tree_method": "hist",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
    }

    booster = xgb.train(params, dtrain, num_boost_round=params["n_estimators"])

    preds = booster.predict(dtrain)
    auc = roc_auc_score(y_train_np, preds)

    return auc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best Params:", best_params)

[I 2025-12-02 10:34:52,397] A new study created in memory with name: no-name-82e58b68-0635-4656-bca8-c39cf9ef7b0f
[W 2025-12-02 10:34:52,481] Trial 0 failed with parameters: {'learning_rate': 0.20365043435693797, 'max_depth': 6, 'min_child_weight': 10, 'subsample': 0.614919701248211, 'colsample_bytree': 0.8283000676636636, 'n_estimators': 511} because of the following error: NameError("name 'dtrain' is not defined").
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-3853281428.py", line 20, in objective
    booster = xgb.train(params, dtrain, num_boost_round=params["n_estimators"])
                                ^^^^^^
NameError: name 'dtrain' is not defined
[W 2025-12-02 10:34:52,500] Trial 0 failed with value None.


NameError: name 'dtrain' is not defined

In [ ]:
# ============================
# 6. TRAIN FINAL MODEL (GPU)
# ============================

best_params.update({
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "tree_method": "gpu_hist",
    "predictor": "gpu_predictor"
})

final_model = xgb.train(best_params, dtrain, num_boost_round=best_params["n_estimators"])

In [ ]:
# ============================
# 7. PREDICT TEST + SAVE SUBMISSION
# ============================

pred_test = final_model.predict(dtest)

submission = pl.DataFrame({
    "TransactionID": test["TransactionID"],
    "isFraud": pred_test
})

submission.write_csv("submission.csv")
print("Done! File saved as submission.csv")


### **Pakai CPU Saja**

In [ ]:
import polars as pl
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
# ==========================
# 1. LOAD DATASET (POLARS)
# ==========================

# train = pl.read_csv("/content/train_transaction.csv")
# test  = pl.read_csv("/content/test_transaction.csv")

# print("Train shape:", train.shape)
# print("Test shape:", test.shape)

In [ ]:
# ==========================
# 2. HANDLE MISSING VALUES
# ==========================

def fill_null(df):
    num_cols = [c for c in df.columns if df[c].dtype.is_numeric()]
    str_cols = [c for c in df.columns if df[c].dtype == pl.String]

    df = df.with_columns([
        pl.col(num_cols).fill_null(0),
        pl.col(str_cols).fill_null("unknown")
    ])

    return df

train_transaction = fill_null(train_transaction)
test_transaction  = fill_null(test_transaction)

In [ ]:
# ==========================
# 3. SPLIT FEATURE & LABEL
# ==========================

# This cell should be renumbered or adjusted if other cells are inserted before it.
# The content of this cell should now be placed after the new Label Encoding cell.


In [ ]:
# ==========================
# 4. TRAIN / VALID SPLIT
# ==========================

# This cell now needs to be renumbered or adjusted if other cells are inserted before it.
# Keeping its original content for now, assuming the user will handle reordering if needed.
# For LightGBM, the conversion to numpy arrays is handled internally from pandas dataframes.


In [ ]:
# ==========================
# 5. OPTUNA TUNING (CPU)
# ==========================

def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "tree_method": "hist",      # CPU saja
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 5),
        "n_estimators": trial.suggest_int("n_estimators", 300, 1200),
    }

    model = xgb.XGBClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False
    )

    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)

    return auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best Params:", best_params)

In [ ]:
# ==========================
# 6. FINAL MODEL (CPU)
# ==========================

final_model = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist"   # tetap CPU
)

final_model.fit(X_np, y_np)

# ==========================
# 7. PREDICT TEST
# ==========================

X_test_np = test[feature_cols].to_numpy()
pred_test = final_model.predict_proba(X_test_np)[:, 1]

submission = pl.DataFrame({
    "TransactionID": test["TransactionID"],
    "isFraud": pred_test
})

submission.write_csv("submission.csv")
print("Selesai. File = submission.csv")


###**2.0**

In [ ]:
# =======================================================
# 1. IMPORT
# =======================================================
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import optuna


In [ ]:
# =======================================================
# 2. LOAD DATA
# =======================================================
# train_df = pd.read_csv("train_transaction.csv")
# test_df  = pd.read_csv("test_transaction.csv")

target = "isFraud"


In [ ]:
# # =======================================================
# # 3. HANDLE MISSING VALUE (PISAH NUM & CAT)
# # =======================================================
# num_cols_train = [c for c in train_transaction.columns if train_transaction[c].dtype.is_numeric()]
# num_cols_test = [c for c in test_transaction.columns if test_transaction[c].dtype.is_numeric()]
# cat_cols = [c for c in train_transaction.columns if train_transaction[c].dtype == pl.String]


# # pastikan target tidak ikut dalam imputasi numeric
# num_cols_train = [c for c in num_cols_train if c != target]

# # numeric → median
# train_transaction[num_cols_train] = train_transaction.with_columns([
#     pl.col(num_cols_train).fill_null(pl.col(num_cols_train).median())
# ])
# test_transaction[num_cols_test] = test_transaction.with_columns([
#     pl.col(num_cols_test).fill_null(pl.col(num_cols_test).median())
# ])

# # categorical → "missing"
# train_transaction[cat_cols] = train_transaction.with_columns([
#     pl.col(cat_cols).fill_null(pl.col(cat_cols).fill_null("unknown"))
# ])
# test_transaction[cat_cols] = test_transaction.with_columns([
#     pl.col(cat_cols).fill_null(pl.col(cat_cols).fill_null("unknown"))
# ])

In [ ]:
# ==========================
# 3. HANDLE MISSING VALUES
# ==========================

def fill_null(df):
    num_cols = [c for c in df.columns if df[c].dtype.is_numeric()]
    str_cols = [c for c in df.columns if df[c].dtype == pl.String]

    df = df.with_columns([
        pl.col(num_cols).fill_null("median"),
        pl.col(str_cols).fill_null("unknown")
    ])

    return df

train_transaction = fill_null(train_transaction)
test_transaction  = fill_null(test_transaction)


In [ ]:
# # =======================================================
# # 4. LABEL ENCODING
# # =======================================================
# encoders = {}
# for col in cat_cols:
#     le = LabelEncoder()
#     combined = pd.concat([train_transaction[col], test_transaction[col]], axis=0)
#     le.fit(combined)
#     train_transaction[col] = le.transform(train_transaction[col])
#     test_transaction[col]  = le.transform(test_transaction[col])
#     encoders[col] = le

NameError: name 'cat_cols' is not defined

In [ ]:
# =======================================================
# 5. SPLIT TRAIN–VALID
# =======================================================
X = train_transaction.drop([target]).to_pandas()
y = train_transaction[target].to_pandas()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# =======================================================
# 6. OPTUNA HYPERPARAMETER TUNING (GPU SUPPORT)
# =======================================================
def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
        "n_estimators": trial.suggest_int("n_estimators", 200, 1200),
        "device": "cpu"
    }

    model = lgb.LGBMClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )

    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)

    return auc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_params["objective"] = "binary"
best_params["metric"] = "auc"
best_params["device"] = "cpu"

[I 2025-12-02 12:27:00,048] A new study created in memory with name: no-name-ba6eac3b-b5c3-4baf-bfbf-e84f58cee67d
[W 2025-12-02 12:27:00,107] Trial 0 failed with parameters: {'feature_fraction': 0.9049928415601843, 'bagging_fraction': 0.6805432549978744, 'bagging_freq': 4, 'min_child_samples': 48, 'num_leaves': 174, 'learning_rate': 0.0070621123188693204, 'n_estimators': 647} because of the following error: ValueError('pandas dtypes must be int, float or bool.\nFields with bad pandas dtypes: ProductCD: object, card4: object, card6: object, P_emaildomain: object, R_emaildomain: object, M1: object, M2: object, M3: object, M4: object, M5: object, M6: object, M7: object, M8: object, M9: object').
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-3098300726.py", line 22, in objective
    model.fit(
  File "/usr/lo

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: ProductCD: object, card4: object, card6: object, P_emaildomain: object, R_emaildomain: object, M1: object, M2: object, M3: object, M4: object, M5: object, M6: object, M7: object, M8: object, M9: object

In [ ]:
# ==========================
# 4. LABEL ENCODING CATEGORICAL FEATURES
# ==========================

# Re-identify string columns after null filling
str_cols_train = [c for c in train_transaction.columns if train_transaction[c].dtype == pl.String]
str_cols_test = [c for c in test_transaction.columns if test_transaction[c].dtype == pl.String]

# Use common string columns to ensure consistency
common_str_cols = list(set(str_cols_train) & set(str_cols_test))

encoders = {}
for col in common_str_cols:
    le = LabelEncoder()
    # Combine train and test data for fitting to ensure consistent encoding
    combined_data = pl.concat([train_transaction.select(col), test_transaction.select(col)]).to_series().to_list()
    le.fit(combined_data)

    # Transform train_transaction
    train_transaction = train_transaction.with_columns(
        pl.Series(name=col, values=le.transform(train_transaction.select(col).to_series().to_list())).alias(col)
    )
    # Transform test_transaction
    test_transaction = test_transaction.with_columns(
        pl.Series(name=col, values=le.transform(test_transaction.select(col).to_series().to_list())).alias(col)
    )
    encoders[col] = le

# print("Train dtypes after encoding:", train_transaction.dtypes)
# print("Test dtypes after encoding:", test_transaction.dtypes)

In [ ]:
# =======================================================
# 7. TRAIN FINAL MODEL
# =======================================================
model = lgb.LGBMClassifier(**best_params)
model.fit(X_train, y_train)

In [ ]:
# =======================================================
# 8. PREDICT TEST
# =======================================================
test_pred = model.predict_proba(test_transaction)[:, 1]

submission = pd.DataFrame({
    "TransactionID": test_transaction["TransactionID"] if "TransactionID" in test_transaction.columns else range(len(test_transaction)),
    "isFraud": test_pred
})
submission.to_csv("submission_lightgbm.csv", index=False)

print("Done: submission_lightgbm.csv")


[I 2025-12-02 12:22:56,833] A new study created in memory with name: no-name-f0d26ce1-1136-42b6-ba7a-206665ce8837
[W 2025-12-02 12:22:56,841] Trial 0 failed with parameters: {'feature_fraction': 0.9526530266387557, 'bagging_fraction': 0.5026681397954519, 'bagging_freq': 4, 'min_child_samples': 123, 'num_leaves': 135, 'learning_rate': 0.0499853677881673, 'n_estimators': 619} because of the following error: TypeError("LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-2782593714.py", line 22, in objective
    model.fit(
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'
[W 2025-12-02 12:22:56,843] Trial 0 failed with value None.


TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'

### **3.0**

In [ ]:
# =======================================================
# 1. IMPORT LIBRARIES
# =======================================================
import polars as pl
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import optuna

In [ ]:
# =======================================================
# 2. LOAD DATASET DENGAN POLARS
# =======================================================
# train_df = pl.read_csv("train_transaction.csv")
# test_df  = pl.read_csv("test_transaction.csv")

target = "isFraud"

In [ ]:
# =======================================================
# 3. IDENTIFIKASI NUMERIC & CATEGORICAL
# =======================================================
num_cols = [
    c for c in train_transaction.columns
    if train_transaction[c].dtype in (pl.Int32, pl.Int64, pl.Float32, pl.Float64) and c != target
]

cat_cols = [
    c for c in train_transaction.columns
    if train_transaction[c].dtype == pl.Utf8
]

In [ ]:
# =======================================================
# 4. HANDLE MISSING VALUES
# =======================================================

# Numeric → 0 (lebih ringan dari median)
train_transaction = train_transaction.with_columns(pl.col(num_cols).fill_null(0))
test_transaction  = test_transaction.with_columns(pl.col(num_cols).fill_null(0))

# Categorical → "missing"
train_transaction = train_transaction.with_columns(pl.col(cat_cols).fill_null("missing"))
test_transaction  = test_transaction.with_columns(pl.col(cat_cols).fill_null("missing"))

In [ ]:
# =======================================================
# 5. LABEL ENCODING (POLARS KE PANDAS LALU KEMBALI)
# =======================================================
# for col in cat_cols:
#     le = LabelEncoder()

#     # gabungkan train + test
#     combined = pl.concat([train_transaction[col], test_transaction[col]]).to_pandas().astype(str)

#     le.fit(combined)

#     train_transaction = train_transaction.with_columns(
#         pl.Series(col, le.transform(train_transaction[col].to_pandas().astype(str)))
#     )
#     test_transaction = test_transaction.with_columns(
#         pl.Series(col, le.transform(test_transaction[col].to_pandas().astype(str)))
#     )


encoders = {}
for col in cat_cols:
    le = LabelEncoder()

    combined = pl.concat([train_transaction[col], test_transaction[col]])
    combined_list = combined.to_list()

    le.fit([str(x) for x in combined_list])

    train_transaction = train_transaction.with_columns(
        pl.Series(col, le.transform([str(x) for x in train_transaction[col].to_list()]))
    )
    test_transaction = test_transaction.with_columns(
        pl.Series(col, le.transform([str(x) for x in test_transaction[col].to_list()]))
    )

    encoders[col] = le

In [ ]:
# =======================================================
# 6. SPLIT FEATURES & LABEL
# =======================================================
feature_cols = [c for c in train_transaction.columns if c != target]

# Polars → list-of-rows → LightGBM compatible
X_list = [list(row) for row in train_transaction.select(feature_cols).iter_rows()]
y_list = train_transaction[target].to_list()

# Train/val split manual tanpa Pandas
X_train, X_val, y_train, y_val = train_test_split(
    X_list, y_list, test_size=0.2, random_state=42, stratify=y_list
)

In [ ]:
# =======================================================
# 7. OPTUNA HYPERPARAMETER TUNING (CPU)
# =======================================================
def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
        "n_estimators": trial.suggest_int("n_estimators", 200, 1200),
        "device": "cpu"     # CPU ONLY
    }

    model = lgb.LGBMClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc",
        early_stopping_rounds=100,
        verbose=False
    )

    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)

    return auc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_params["objective"] = "binary"
best_params["metric"] = "auc"
best_params["device"] = "cpu"

[I 2025-12-02 12:54:17,089] A new study created in memory with name: no-name-af0d147a-affa-487c-8637-ef73e7135ef2
[W 2025-12-02 12:54:17,155] Trial 0 failed with parameters: {'feature_fraction': 0.7747227360166218, 'bagging_fraction': 0.6001635626522509, 'bagging_freq': 2, 'min_child_samples': 24, 'num_leaves': 211, 'learning_rate': 0.0192007127007574, 'n_estimators': 1049} because of the following error: TypeError("LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-2098952864.py", line 22, in objective
    model.fit(
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'
[W 2025-12-02 12:54:17,177] Trial 0 failed with value None.


TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'

In [ ]:
# =======================================================
# 8. TRAIN FINAL MODEL
# =======================================================
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X, y)


# =======================================================
# 9. PREDICT ON TEST SET
# =======================================================
X_test = test_df.select(feature_cols).to_numpy()
test_pred = final_model.predict_proba(X_test)[:, 1]

submission = pl.DataFrame({
    "TransactionID": test_df["TransactionID"] if "TransactionID" in test_df.columns else pl.Series(range(len(test_df))),
    "isFraud": test_pred
})

submission.write_csv("submission_lightgbm_polars.csv")
print("Done! Saved as submission_lightgbm_polars.csv")


### **4.0**

In [ ]:
!pip install optuna-integration[lightgbm]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade lightgbm optuna

In [ ]:
# ======================================================
# 1. IMPORT LIBRARIES
# ======================================================
import polars as pl
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import optuna
import os
import optuna.integration.lightgbm as lgb_tuner

# from optuna.integration import lightgbm as lgb_tuner
# from lightgbm import LightGBMTunerCV
from sklearn.model_selection import StratifiedKFold


In [ ]:
# ======================================================
# 2. LOAD DATASET WITH POLARS
# ======================================================
# train_transactions = pl.read_csv("train_transaction.csv")
# test_transactions  = pl.read_csv("test_transaction.csv")

target = "isFraud"

In [ ]:
# =======================================================
# 3. SPLIT NUMERIC & CATEGORICAL COLUMNS & IMPUTATION
# =======================================================
num_cols = [c for c, t in zip(train_transaction.columns, train_transaction.dtypes)
            if t in (pl.Float32, pl.Float64, pl.Int64, pl.Int32) and c != target]

cat_cols = [c for c, t in zip(train_transaction.columns, train_transaction.dtypes)
            if t == pl.Utf8]

# numeric → median
for col in num_cols:
    med = train_transaction[col].median()
    train_transaction = train_transaction.with_columns(pl.col(col).fill_null(med))
    test_transaction  = test_transaction.with_columns(pl.col(col).fill_null(med))

# categorical → missing
for col in cat_cols:
    train_transaction = train_transaction.with_columns(pl.col(col).fill_null("missing"))
    test_transaction  = test_transaction.with_columns(pl.col(col).fill_null("missing"))

In [ ]:
# =======================================================
# 4. LABEL ENCODING (HANYA POLARS)
# =======================================================
for col in cat_cols:
    # Ambil seluruh nilai unik
    unique_vals = (
        pl.concat([train_transaction[col], test_transaction[col]])
        .unique()
        .to_list()
    )

    # Buat mapping integer
    mapping = {v: i for i, v in enumerate(unique_vals)}

    # Terapkan ke train & test (Polars)
    train_transaction = train_transaction.with_columns(
        pl.col(col).replace(mapping).cast(pl.Int64)
    )
    test_transaction = test_transaction.with_columns(
        pl.col(col).replace(mapping).cast(pl.Int64)
    )

In [ ]:
# ======================================================
# 5. SAVE CLEANED DATA TO CSV (FOR STREAMING MODE)
# ======================================================

# train_transaction.write_csv("train_clean.csv")
# test_transaction.write_csv("test_clean.csv")

# # Save labels separately (LightGBM needs it)
# y_list = train_transaction[target].to_list()
# pl.DataFrame({"isFraud": y_list}).write_csv("train_label.csv")

# feature_cols = [c for c in train_transaction.columns if c != target]


# train_transaction = train_transaction.sample(frac=1.0, shuffle=True)

# val_size = int(0.2 * train_transaction.height)

# val_df = train_transaction[:val_size]
# train_transaction2 = train_transaction[val_size:]

# X_train = train_transaction2.drop(target).to_numpy()
# y_train = train_transaction2[target].to_numpy()

# X_val   = val_df.drop(target).to_numpy()
# y_val   = val_df[target].to_numpy()



# train_lgb = lgb.Dataset(X_train, label=y_train)
# valid_lgb = lgb.Dataset(X_val, label=y_val)

In [ ]:
# ======================================================
# 6. LIGHTGBM STREAMING DATASET
# ======================================================

# train_data = lgb.Dataset(
#     "train_clean.csv",
#     label="train_label.csv",
#     params={
#         "header": True,
#         "label_column": "isFraud",
#     }
# )

In [ ]:
# print(train_transaction["isFraud"].dtype)
# print(train_transaction.dtypes)

Int64
[Float64, Int64, Float64, Float64, Int64, Float64, Float64, Float64, Int64, Float64, Int64, Float64, Float64, Float64, Float64, Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float6

In [ ]:
# =======================================================
# 5. CONVERT TO LIGHTGBM DATASET ONCE (ANTI-CRASH)
# =======================================================
X = train_transaction.drop(target).to_pandas()  # safe (done once)
y = train_transaction[target].to_pandas()

train_lgb = lgb.Dataset(X, label=y)

In [ ]:
# ======================================================
# 7. OPTUNA TUNING (STREAMING MODE)
# ======================================================
# def objective(trial):
#     params = {
#         "objective": "binary",
#         "metric": "auc",
#         "boosting_type": "gbdt",
#         "verbosity": -1,
#         "num_leaves": trial.suggest_int("num_leaves", 31, 255),
#         "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
#         "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
#         "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
#         "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
#         "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
#         "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
#         "device": "cpu"
#     }

#     model = lgb.train(params, train_data)

#     # Evaluate on a small validation set (to keep RAM low)
#     val_sample = train_transaction.sample(50000)
#     X_val = val_sample.select(feature_cols).to_pandas()
#     y_val = val_sample[target].to_pandas()

#     preds = model.predict(X_val)
#     auc = roc_auc_score(y_val, preds)

#     return auc


# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)  # keep small for safety

# best_params = study.best_params
# best_params.update({
#     "objective": "binary",
#     "metric": "auc",
#     "device": "cpu"
# })

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

tuner = lgb_tuner.LightGBMTunerCV(
    params,
    train_lgb,
    folds=folds,
    # fit_kwargs={
    #     "early_stopping_rounds": 50,
    #     "verbose_eval": False  # <-- pindahkan ke sini
    # }
)

tuner.run()  # <<== OPTUNA tidak crash karena LGBMTuner sangat stabil

best_params = tuner.best_params
best_score = tuner.best_score

print("Best params:", best_params)
print("Best AUC:", best_score)

[I 2025-12-02 14:30:12,034] A new study created in memory with name: no-name-ebadb0b4-fe64-4eba-bf71-b78b8576a864
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:877: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
feature_fraction, val_score: 0.962358:  14%|#4        | 1/7 [16:52<1:41:16, 1012.76s/it]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:877: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
feature_fraction, val_score: 0.962358:  29%|##8       | 2/7 [37:28<1:35:18, 1143.77s/it]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:877: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(
feature_fraction, val_score: 0.962358:  43%|####2     | 3/7 [1:09:29<1:39:55, 1498.92s/it]/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_spl

In [ ]:
# ======================================================
# 8. TRAIN FINAL MODEL (STREAMING MODE)
# ======================================================
final_model = lgb.train(best_params, train_lgb)

In [ ]:
X_test = test_transaction.drop("TransactionID").to_pandas()
test_pred = final_model.predict(X_test)

submission = pl.DataFrame({
    "TransactionID": test_transaction["TransactionID"],
    "isFraud": test_pred
})

submission.write_csv("submission_done.csv")
print("Done! Saved as submission_done.csv")
